In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
import gc
import os
import matplotlib.pyplot as plt
import seaborn as sns 
import lightgbm as lgb
import itertools
import pickle, gzip
import glob
from sklearn.preprocessing import StandardScaler
from tsfresh.feature_extraction import extract_features
np.warnings.filterwarnings('ignore')
import dask.dataframe as dd
import missingno as msno
from pandasql import sqldf
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import KFold
import matplotlib.gridspec as gridspec
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

In [2]:
#Always seed the randomness of this universe
np.random.seed(51)

In [3]:
%%time
pred1 = dd.read_csv('test_pred_32.csv')
pred2 = dd.read_csv('test_pred_33.csv')
pred1 = pred1.compute()
pred2 = pred2.compute()
print(pred1.shape,pred2.shape)

(3492890, 16) (3492890, 16)
CPU times: user 32.4 s, sys: 2.94 s, total: 35.4 s
Wall time: 8.61 s


In [4]:
%%time
pred1 = pred1.reset_index(drop=True)
pred2 = pred2.reset_index(drop=True)

CPU times: user 191 ms, sys: 265 ms, total: 456 ms
Wall time: 454 ms


In [5]:
pred1['object_id'].equals(pred2['object_id'])

True

In [6]:
%%time
test_ensemble = pd.DataFrame()

CPU times: user 397 µs, sys: 47 µs, total: 444 µs
Wall time: 452 µs


In [7]:
coef_lgb = 0.85
coef_mlp = 0.15

In [8]:
%%time
test_ensemble = pred1.iloc[:,1:]*coef_lgb + pred2.iloc[:,1:]*coef_mlp

CPU times: user 292 ms, sys: 609 ms, total: 901 ms
Wall time: 899 ms


In [9]:
print(test_ensemble.shape)

(3492890, 15)


In [10]:
test_ensemble['object_id'] = pred1['object_id']
print(test_ensemble.shape)

(3492890, 16)


In [11]:
test_ensemble = test_ensemble[list(pred1.columns)]

In [12]:
test_ensemble.tail()

,object_id,class_6,class_15,class_16,class_42,class_52,class_53,class_62,class_64,class_65,class_67,class_88,class_90,class_92,class_95,class_99
3492885,130787966,0.000243,0.360808,0.000450,0.197590,0.132574,0.000516,0.105283,0.000267,0.002158,0.016546,0.002678,0.179840,0.000347,0.000701,0.245522
3492886,130787971,0.000252,0.010714,0.000273,0.044627,0.236696,0.000338,0.018010,0.329932,0.000451,0.022253,0.001048,0.331432,0.000318,0.003656,0.244092
3492887,130787974,0.000677,0.018175,0.000176,0.896960,0.022306,0.000407,0.047582,0.000068,0.000212,0.001203,0.000446,0.009917,0.000280,0.001592,0.077449
3492888,130788053,0.000098,0.699977,0.000156,0.139274,0.004244,0.000186,0.010052,0.109660,0.000542,0.000477,0.000332,0.033885,0.000160,0.000956,0.169347
3492889,130788054,0.000292,0.099141,0.000396,0.467836,0.125158,0.000780,0.055222,0.000393,0.001035,0.022333,0.001994,0.223577,0.000858,0.000985,0.226800


In [13]:
%%time
test_ensemble.to_csv('test_pred_36.csv',index=False)

CPU times: user 1min 26s, sys: 1.02 s, total: 1min 27s
Wall time: 1min 28s


In [14]:
test_ensemble['object_id'].equals(pred1['object_id'])

True

In [15]:
!kaggle competitions submit -c PLAsTiCC-2018 -f test_pred_36.csv -m "Message"

100%|█████████████████████████████████████| 1.06G/1.06G [2:10:16<00:00, 146kB/s]
Successfully submitted to PLAsTiCC Astronomical Classification